In [ ]:
from bottle import route, run, request, response
import json
from datetime import datetime

# Simulated email status database
email_status_db = {
    1: {"email": "user1@example.com", "status": "Sent", "timestamp": "2024-11-15 09:00:00"},
    2: {"email": "user2@example.com", "status": "Pending", "timestamp": "2024-11-15 10:00:00"},
    3: {"email": "user3@example.com", "status": "Failed", "timestamp": "2024-11-15 11:00:00"},
}

@route('/emails', method='GET')
def get_email_statuses():
    """Retrieve all email statuses."""
    response.content_type = 'application/json'
    return json.dumps(email_status_db)

@route('/emails/<email_id:int>', method='GET')
def get_email_status(email_id):
    """Retrieve the status of a specific email by ID."""
    if email_id in email_status_db:
        response.content_type = 'application/json'
        return json.dumps(email_status_db[email_id])
    else:
        response.status = 404
        return {"error": "Email not found"}

@route('/emails', method='POST')
def add_email():
    """Add an email to the database for sending."""
    try:
        data = request.json
        email = data.get("email")
        if not email:
            response.status = 400
            return {"error": "Email field is required"}

        email_id = max(email_status_db.keys()) + 1
        email_status_db[email_id] = {
            "email": email,
            "status": "Pending",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        }
        response.status = 201
        return {"message": "Email added successfully", "email_id": email_id}
    except Exception as e:
        response.status = 500
        return {"error": str(e)}

@route('/emails/<email_id:int>', method='PUT')
def update_email_status(email_id):
    """Update the status of an email."""
    try:
        if email_id not in email_status_db:
            response.status = 404
            return {"error": "Email not found"}

        data = request.json
        status = data.get("status")
        if not status:
            response.status = 400
            return {"error": "Status field is required"}

        email_status_db[email_id]["status"] = status
        email_status_db[email_id]["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return {"message": "Email status updated successfully"}
    except Exception as e:
        response.status = 500
        return {"error": str(e)}

run(host='localhost', port=8080)
